# Grain Size Distribution - Height Correlation Function

Find relevant parameters describing the topography of sputtered Ruthenium thin films on Si substrate

How:
- Roughness RMS
- Fit exponential increase
- Take intersection between RMS asymptote and Fit

Output parameters:
- Roughness
- Correlation Length
- Alpha exponent

Note: here I am using the images which have their contrast adjusted. For reference:
http://homepages.inf.ed.ac.uk/rbf/HIPR2/stretch.htm

In [1]:
import os
import sys
import json

import numpy as np
import pandas as pd
import pickle

%matplotlib nbagg
%reload_ext autoreload
%autoreload 2
import matplotlib
import matplotlib.lines as mlines
import matplotlib.pyplot as plt

from matplotlib import cm
from skimage import data, io, exposure, img_as_float

from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter

In [2]:
debug = True #or 'False' when you do not want to debug

path = "/Users/sfiligoj/Desktop/SEM_all_contrasted_corrected/"
os.chdir(path)
filess = os.listdir()
filess = [f for f in filess if (f.endswith('tif') and '*' not in f)]
print(f"Found {len(filess)} files")

Found 151 files


In [3]:
# read-in the dictionary of Scan Size values:
# (see Manual Reading FOV file)
with open('FOVs.json', 'r') as f:
    FOV_dict = json.loads(f.read())

In [6]:
reference = io.imread('Ru_LH_700C_075.tif', as_gray=True)
reference = img_as_float(reference[:1024,:])
# We create the empty DataFrame:
df = pd.DataFrame(
        columns= ['filename', 'density', 'temp', 'hhcorr', 'scansize', 'n_pix'])

for i,file in enumerate(filess[:]):
    print(f'running file: {i}')
    
    # Parse information from filename
    if 'L_' in file:
        density = 'L'
        if 'asdep' in file:
            temperature = '25'
        else:
            temperature = file[2:5]
        
    if 'H_' in file:
        density = 'H'
        if 'asdep' in file:
            temperature = '25'
        else:
            temperature = file[2:5]

    if 'Ru_LH' in file:
        density = 'LH'
        if 'asdep' in file:
            temperature = '25'
        else:
            temperature = file[6:9]
    
    print(f'density: {density} and temperature {temperature}')

    img = io.imread(file, as_gray = True)
    img = img_as_float(img[:1024,:])
    
    
    # Contrast correction:   
    img = exposure.match_histograms(img, reference)
    
    # Fixed parameters:  
    N_Pixel = img.shape[0]                              # before was len(data), which works in this case, but it would be the N of rows 
    ScanSize = FOV_dict[file]
    StepSize = ScanSize/N_Pixel                          # nm/px     
        
    Xdelta = np.linspace(StepSize, ScanSize, num=N_Pixel)
    
    
    # ***    Calculate Height-Height correlation function:   ***

    HHcorr = np.zeros(N_Pixel)
    for px_dist in range(0, N_Pixel-1):
        shifted_data = img[:,px_dist+1:]
        data_section = img[:,:-px_dist-1]
        difference = (data_section-shifted_data)**2
        HHcorr[px_dist] = np.mean(difference)
    # The last point of HHcorr is always 0, so we work around it:
    HHcorr[-1] = HHcorr[-2]
    
    df = df.append( 
            {'filename': file , 
             'density': density, 
             'temp': temperature, 
             'hhcorr': HHcorr , 
             'scansize': ScanSize , 
             'n_pix': N_Pixel, 
            },
            ignore_index=True)
    

running file: 0
density: H and temperature 25
running file: 1
density: LH and temperature 500
running file: 2
density: LH and temperature 700
running file: 3
density: L and temperature 25
running file: 4
density: H and temperature 700
running file: 5
density: H and temperature 700
running file: 6
density: LH and temperature 700
running file: 7
density: L and temperature 25
running file: 8
density: LH and temperature 700
running file: 9
density: LH and temperature 500
running file: 10
density: L and temperature 700
running file: 11
density: H and temperature 25
running file: 12
density: LH and temperature 600
running file: 13
density: H and temperature 25
running file: 14
density: H and temperature 800
running file: 15
density: L and temperature 700
running file: 16
density: H and temperature 250
running file: 17
density: L and temperature 25
running file: 18
density: LH and temperature 800
running file: 19
density: H and temperature 700
running file: 20
density: LH and temperature 800


In [7]:
# Save a dictionary into a pickle file.
pickle.dump(df, open( "save.p", "wb" ) )
df

,filename,density,temp,hhcorr,scansize,n_pix
0,H_asdep_006.tif,H,25,"[0.029532958133807113, 0.03172089640968673, 0....",345,1024
1,Ru_LH_500C_015.tif,LH,500,"[0.01841470656196632, 0.02307507579548702, 0.0...",829,1024
2,Ru_LH_700C_077.tif,LH,700,"[0.005772947324770799, 0.006301168366615295, 0...",414,1024
3,L_asdep_007.tif,L,25,"[0.04142936148330632, 0.04775319104077777, 0.0...",11800,1024
4,H_700C_024.tif,H,700,"[0.02107438041374748, 0.02270302762767064, 0.0...",518,1024
...,...,...,...,...,...,...
146,H_500C_5000nm_032.tif,H,500,"[0.024703719096106684, 0.03423447114183298, 0....",6380,1024
147,Ru_LH_800C_033.tif,LH,800,"[0.004526015936482241, 0.0059289094323921095, ...",1180,1024
148,Ru_LH_800C_027.tif,LH,800,"[0.0038333631593081077, 0.0041651386865076065,...",345,1024
149,H_700C_10um_043.tif,H,700,"[0.036170526296200475, 0.0433645346340971, 0.0...",16600,1024


In [ ]:
import pandas_profiling
from pathlib import Path
profile = df.profile_report(title="SEMdata")
profile.to_file(output_file=Path("./SEMdata.html"))